In [59]:
import selenium
import time
from selenium import webdriver as wb
webD=wb.Chrome('chromedriver.exe')
webD.get('https://www.webmotors.com.br/carros/estoque/audi/a3?tipoveiculo=carros&marca1=AUDI&modelo1=A3')

In [60]:
def scroll_to_bottom(driver):

    old_position = 0
    new_position = None

    while new_position != old_position:
        # Get old scroll position
        old_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))
        # Sleep and Scroll
        time.sleep(1)
        driver.execute_script((
                "var scrollingElement = (document.scrollingElement ||"
                " document.body);scrollingElement.scrollTop ="
                " scrollingElement.scrollHeight;"))
        # Get new position
        new_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))
    
    try:
        load_more = webD.find_element_by_id('ButtonCarriesMoreCars')
        load_more.click()
        return False
    except:
        return True

In [61]:
#page_end = False
#while page_end == False:
#    page_end = scroll_to_bottom(webD)

In [62]:
carsList = webD.find_elements_by_class_name('sc-dNLxif.dcKUo')

In [63]:
carsList[0].find_element_by_tag_name('a').get_property('href')

'https://www.webmotors.com.br/comprar/audi/a3/2-0-tfsi-sportback-16v-gasolina-4p-s-tronic/4-portas/2007-2008/28030416?pos=a28030416c:&np=1'